<a href="https://colab.research.google.com/github/pszemraj/ai-msgbot/blob/main/colab-notebooks/GPT2_general_conv_textgen_training_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  aitextgen — Train a GPT-2 (or GPT Neo) Text-Generating Model w/ GPU

This notebook is based on the original tutorial from `aitextgen`!

- For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).
- for `ai-msgbot` (which is using `aitextgen` for chatbot-esque purposes) you can find the project repo [here](https://github.com/pszemraj/ai-msgbot)


_updates made by [Peter](https://peterszemraj.ch/)_



---

## GPU

Colaboratory uses a Nvidia K80, an Nvidia P100, an Nvidia V100, or Nvidia A100. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a k80 or a P100 is ideal since they have more VRAM. 

- In theory: **If you receive a T4 or a V100 GPU, you can enable `fp16=True` during training for faster/more memory efficient training.**

In [1]:
!nvidia-smi

Tue Nov 23 04:23:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    31W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## formatting

In [2]:
from IPython.display import HTML, display
# colab formatting
def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )

get_ipython().events.register("pre_run_cell", set_css)

# setup

In [3]:
# update torch in case using a A100 GPU
# may take 5+ minutes
!pip3 install torch==1.10.0+cu113 torchvision==0.11.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html -q

In [4]:
!pip install -q aitextgen

import logging

logging.basicConfig(
    format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

In [5]:
mount_gdrive()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading GPT-2 or GPT Neo


- A common use case is *continuing* to fine-tune a model that was originally pretrained, and then fine-tuned a little bit, but needs to be fine-tuned more for accuracy/saliency reasons or because Google cut off the runtime earlier. 
    - in this case, `load_from_folder` should be set to `True` and `load_folder_dir` points to where the model checkpoint is on your google drive.
- **the below section describes loading an new/pretrained model from the original tutorial.**

> If you're retraining a model on new text, you need to download and load the GPT-2 model into the GPU. 

> There are several sizes of GPT-2:

    * `124M` (default): the "small" model, 500MB on disk.
    * `355M` (default): the "medium" model, 1.5GB on disk.
    * `774M` (default): the "large" model, 3GB on disk.

> You can also finetune a GPT Neo model instead ([_or any textgen GPT-architecture model on huggingface_](https://huggingface.co/models?pipeline_tag=text-generation)), which is more suitable for longer texts and the base model has more recent data:

* `125M`: Analogous to the GPT-2 124M model. (355M parameter model was removed)
*  `EleutherAI/gpt-neo-1.3B` : 1.3 billion parameter model. Have yet to see this train on Colab without crashing

> The next cell downloads the model and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

## 

In [6]:
model_size = "355M" #@param ["355M", "774M"]
load_from_folder = False #@param {type:"boolean"}
load_folder_dir = "/content/drive/MyDrive/Programming/AI_peter/gpt2-25KtriviaQA100KnaturalQA50Kdailydialogues-774M" #@param {type:"string"}


In [7]:
if load_from_folder:
    ai = aitextgen(model_folder=load_folder_dir, to_gpu=True,
                   gradient_checkpointing=True)
else:
    ai = aitextgen(tf_gpt2=model_size, to_gpu=True,
                   gradient_checkpointing=True)
# Comment out the above line and uncomment the below line to use GPT Neo instead.
# ai = aitextgen(model='distilgpt2', 
#                to_gpu=True, 
#                gradient_checkpointing=True)

11/23/2021 04:24:00 — INFO — aitextgen — Loading 355M GPT-2 model from /aitextgen.
11/23/2021 04:24:04 — INFO — aitextgen — GPT2 loaded with 354M parameters.
11/23/2021 04:24:04 — INFO — aitextgen — Gradient checkpointing enabled for model training.
11/23/2021 04:24:04 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## load training data


- <font color="orange"> combine any other data from the a "standard" conversational dataset and do a final pass of training (with different data) </font>

In [8]:
dl_link = "https://github.com/pszemraj/ai-msgbot/raw/main/conversation-data/wizard-of-wikipedia/ScriptParse-wow-train-kilt.txt" #@param {type:"string"}


In [9]:
# download test image
from urllib import request
from os.path import join
import os
vm_wd = os.getcwd()
local_name = join(vm_wd, "training_script.txt")
request.urlretrieve(dl_link, local_name)


('/content/training_script.txt', <http.client.HTTPMessage at 0x7fb6973f4c10>)

In [10]:
# adjust names in script if needed  
import pprint as pp
from os.path import basename

def update_script_names(local_name, spkr_from="speaker a", spkr_to="john smith",
                        resp_from="speaker b", resp_to="nancy sellers",
                        verbose=False):
    """
    update_script_names - if the textfile script has different names for the 
    speaker/responder than desired (i.e. it is a group conversation, and the 
    chatbot is just supposed to simulate 1:1) this function can be used to 
    standardize
    """

    with open(local_name, 'r', encoding='utf-8', errors='ignore') as fi:
        orig_lines = fi.readlines()

    from tqdm.auto import tqdm

    upd_lines = []

    for line in tqdm(orig_lines, total=len(orig_lines), 
                     desc="replacing speaker names"):
        
        fixline = line.replace(spkr_from, spkr_to)
        fixline = fixline.replace(resp_from, resp_to)
        upd_lines.append(fixline)

    local_namev2 = join(vm_wd, "V2-rename-" + basename(local_name))

    with open(local_namev2, 'w', encoding='utf-8', errors='ignore') as fo:
        fo.writelines(upd_lines)

    if verbose: pp.pprint(upd_lines[:10])
    # return filepath
    return local_namev2


In [11]:
file_name = local_name # update if using fn above

# Train / Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`fp16`**: Enables half-precision training for faster/more memory-efficient training. Only works on a T4 or V100 GPU.

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM. (if using `fp16`, you can increase the batch size more safely)

In [12]:
base_dir = "/content/drive/MyDrive/Programming/ai-msgbot" #@param {type:"string"}
# update to yours

In [13]:
import gc, os
from os.path import join
from datetime import datetime

def get_timestamp():
    return datetime.now().strftime("%b-%d-%Y_t-%H")

temp_gpu_path = join(base_dir, 
                     "GPT2-conversational-{sz}-{dt}".format(sz=model_size,
                                                            dt=get_timestamp(),
                                                            )
                     )
os.makedirs(temp_gpu_path, exist_ok=True)
gc.collect()

302

## example outputs

- this cell had its outputs cleared before notebook was posted.
- the below is an example of what the outputs during training should look like.

```
1,000 steps reached: saving model to //content/drive/MyDrive/Programming/ai-msgbot/GPT2-conversational-355M-Nov-23-2021_t-04
1,500 steps reached: generating sample texts.
/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
==========
.

person alpha:
i love to read. reading. reading requires creativity. it's one of the intricacies of decoding symbols and improving knowledge, sharing information through wires and wires.
is reading still a good thing to read?
not really. reading software allows creativity, refinement and development. it's also a great tool for sharing information and sharing information over the past.
can you give example of reading strategies?

person beta:
its is. reading produces creativity. it's my favorite form of reading, it's used on high shelfs.

person alpha:
do you like to eat at a restaurant? i'm a big fan of cooking and watching my favorite meal.
i love cooking, it is the combination of grilling over an open fire.
i love cooking, it is so good! it's my favorite meal of the day. i like to add to my food in my house.
it is a great meal too, i like to add many different ways to cooking.
i like to add different ways to cooking.
i agree, it is the most common type of cooking.

person beta:
i like to add different techniques and ingredients and ingredients.

person alpha:

==========
2,000 steps reached: saving model to //content/drive/MyDrive/Programming/ai-msgbot/GPT2-conversational-355M-Nov-23-2021_t-04
3,000 steps reached: saving model to //content/drive/MyDrive/Programming/ai-msgbot/GPT2-conversational-355M-Nov-23-2021_t-04
3,000 steps reached: generating sample texts.
==========


person alpha:
i love to cook. cooking is a great art. you can bake!
i love cooking too. cooking as a whole is an art.
```

## t r a i n

In [ ]:
# DO NOT USE WARMUP STEPS

ai.train(file_name,
         output_dir=temp_gpu_path, # where it saves during "save_every"
         line_by_line=False, # if using CSV file input
         from_cache=False,
         num_steps=100000, # takes about 5 hours on 16 gb v100 GPU for 75000
         generate_every=1500,
         max_grad_norm=0.5,
         save_every=1000,
         gradient_accumulation_steps=1,
         save_gdrive=False, # this is an "automated" save which is worse than current method (IMO)
         learning_rate=1e-3,
        #  fp16=True, # may be relevant to set to false (even if available) for "final" training
         batch_size=1, # if pushing model_size you probably want to leave this at 1
        #  fp16_opt_level="O2", # different types of FP16 are possible
        #  amp_backend='apex'
         )

In [29]:
save_path = join(base_dir, 
                     "FINAL-GPT2-conv-{sz}-{dt}".format(sz=model_size,
                                                        dt=get_timestamp(),
                                                        )
                     )

In [30]:
import os
os.makedirs(save_path, exist_ok=True)
ai.save(save_path)

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.


# Use a Trained Model for Generation

If you already had a trained model from this notebook, running the next cell will copy the `pytorch_model.bin` and the `config.json`file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [ ]:
!nvidia-smi

In [ ]:
mount_gdrive()


In [31]:
# best model thus far @ 1.3B parameters and tuned for 50k steps
# from_folder = "/content/drive/MyDrive/Programming/AI_peter/GPT-Neo-1B-V1"

from_folder = save_path

if len(from_folder) > 2:

    for file in ["pytorch_model.bin", "config.json"]:
        if from_folder:
            copy_file_from_gdrive(file, from_folder)
        else:
            copy_file_from_gdrive(file)

    ai = aitextgen(model_folder=from_folder, to_gpu=True)
else:
    ai = aitextgen(model_folder=".", to_gpu=True)


11/23/2021 17:10:09 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/MyDrive/Programming/ai-msgbot/FINAL-GPT2-conv-355M-Nov-23-2021_t-17.
11/23/2021 17:10:13 — INFO — aitextgen — GPT2 loaded with 354M parameters.
11/23/2021 17:10:13 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## Generate Text


`generate()` without any parameters generates a single text from the loaded model to the console.

In [ ]:
ai.generate(n=3, max_length=256, 
            temperature=1.0, top_p=0.9)

In [ ]:
ai.generate(prompt="these days, it always seems like ", temperature=1,
            min_length=10, batch_size =20)

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`min length`**: The minimum length of the generated text: if the text is shorter than this value after cleanup, aitextgen will generate another one.
*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2 and 2048 with GPT Neo)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

In [ ]:
ai.generate(
    n=3, batch_size=25, prompt="i just", max_length=256, 
    temperature=1.0, top_p=0.9
)

For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [27]:
save_loc = join(base_dir, "generated_text_out")

os.makedirs(save_loc, exist_ok=True)

In [ ]:
p_list = [["nancy powell:"+"\n", 
           "how are you doing?"+"\n", "\n", 
           "john smith:" + "\n"], 
           ["nancy powell:"+"\n", "why is "],
           ["john smith:" + "\n"],
]


prompts = ["".join(line) for line in p_list]

In [ ]:
from datetime import datetime
import pprint as pp

ds_date_time = datetime.now().strftime("%m.%d.%Y")

base_header = "gpt-dailydialogue-textgen-{}".format(ds_date_time)
prompt_IDs = [base_header + "_file-{}.txt".format(i+1) for i in range(5, len(prompts)+11)]

prompt_mng = {}
for pid, text in zip(prompt_IDs, prompts):
    prompt_mng[pid] = text
pp.pprint(prompt_mng)

In [28]:
from os.path import join

for pfile, my_prompt in prompt_mng.items():
    ai.generate_to_file(
        n=50,
        batch_size=5,
        prompt=my_prompt,
        max_length=512,
        temperature=0.8,
        top_p=0.9,
        destination_path=join(save_loc, pfile)
    )


11/23/2021 17:02:37 — INFO — aitextgen — Generating 50 texts to /content/drive/MyDrive/Programming/ai-msgbot/generated_text_out/gpt-dailydialogue-textgen-11.23.2021_file-6.txt


  0%|          | 0/50 [00:00<?, ?it/s]

11/23/2021 17:04:56 — INFO — aitextgen — Generating 50 texts to /content/drive/MyDrive/Programming/ai-msgbot/generated_text_out/gpt-dailydialogue-textgen-11.23.2021_file-7.txt


  0%|          | 0/50 [00:00<?, ?it/s]

11/23/2021 17:07:22 — INFO — aitextgen — Generating 50 texts to /content/drive/MyDrive/Programming/ai-msgbot/generated_text_out/gpt-dailydialogue-textgen-11.23.2021_file-8.txt


  0%|          | 0/50 [00:00<?, ?it/s]